## Assignment #1 DATA 730

This assignment tests that you have the course tech stack set up. If you do, you will be able to complete this notebook and turn-in your work by pushing to your assignment repository.

## Tech stack checklist

*Fill out the checklist below once you have completed each task:*

- [x] GitHub account
- [x] SageMaker Studio Lab account

**Create the `data730` conda environment by right clicking the `environment.yml` file from the repo in SageMaker Studio Lab and selecting `Build conda Environment`.** 

In [1]:
# uncomment the following line and execute this cell to demonstrate you have created the data730 conda environment
system('conda env list | grep myenv', intern = TRUE)

[1] "myenv                  /opt/conda/envs/myenv"

In [4]:
# uncomment the following line and execute this cell to demonstrate that you have installed the tidymodels package
library('tidymodels')

── Attaching packages ────────────────────────────────────── tidymodels 1.2.0 ──

✔ broom        1.0.7     ✔ recipes      1.1.0
✔ dials        1.3.0     ✔ rsample      1.2.1
✔ dplyr        1.1.4     ✔ tibble       3.2.1
✔ ggplot2      3.5.1     ✔ tidyr        1.3.1
✔ infer        1.0.7     ✔ tune         1.2.1
✔ modeldata    1.4.0     ✔ workflows    1.1.4
✔ parsnip      1.2.1     ✔ workflowsets 1.1.0
✔ purrr        1.0.2     ✔ yardstick    1.3.1

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ purrr::discard() masks scales::discard()
✖ dplyr::filter()  masks stats::filter()
✖ dplyr::lag()     masks stats::lag()
✖ recipes::step()  masks stats::step()
• Use suppressPackageStartupMessages() to eliminate package startup messages



***

## Let's do some statistical inference!
<a id="org_exp"></a>

Read [this article](https://www.theguardian.com/world/2002/jan/04/euro.eu2). 

Execute the code below to investigate whether the coin is fair...

In [1]:
# These functions simulate tossing a fair coin n_tosses number of times for n_iterations number of iterations

toss_fair_coin_n_times = function(n_tosses = 250) {
    result = ifelse(runif(n = n_tosses) < 0.5, 'heads', 'tails')
    result
}

repeat_the_experiment = function(n_iterations, n_tosses = 250) {
    n_heads_each_iteration = c()
    for (i in 1:n_iterations) {
        n_heads = sum(toss_fair_coin_n_times(n_tosses) == 'heads')
        n_heads_each_iteration = c(n_heads_each_iteration, n_heads)
    } 
    n_heads_each_iteration
}

In [221]:
# Let's do 10,000 experiments of 250 tosses and see how often our outcome deviates from what was actually observed (139 heads)

n_iterations = 10000
n_tosses = 250
observed_deviation = 139 - 125 # 139 heads observed, expected value for a fair coin of 125

n_heads_each_experiment = repeat_the_experiment(n_iterations, n_tosses)

x = mean(abs(n_heads_each_experiment - 125) >= observed_deviation)
paste0('With a fair coin we would expect a deviation as large or larger than what we observed ', x*100, '% of times')

[1] "With a fair coin we would expect a deviation as large or larger than what we observed 8.28% of times"

***

<a id="interp1"></a>

In [222]:
# 125 (this doesn't divide evenly by 2 but is here to investigate the relationship of scaling the number of coin tosses) 
# Let's do 10,000 experiments of 125 tosses and see how often our outcome deviates from what was actually observed (69.5 heads)

n_iterations = 10000
n_tosses = 125
observed_deviation = 139/2 - 125/2 # 139 heads observed, expected value for a fair coin of 125

n_heads_each_experiment = repeat_the_experiment(n_iterations, n_tosses)

x = mean(abs(n_heads_each_experiment - 125/2) >= observed_deviation)
paste0('With a fair coin we would expect a deviation as large or larger than what we observed ', x*100, '% of times')

[1] "With a fair coin we would expect a deviation as large or larger than what we observed 20.73% of times"

<a id="extrap1"></a>

In [173]:
# 500
# Let's do 10,000 experiments of 500 tosses and see how often our outcome deviates from what was actually observed (278 heads)

n_iterations = 10000
n_tosses = 250*2
observed_deviation = 139*2 - 125*2 # 278 heads observed, expected value for a fair coin of 250

n_heads_each_experiment = repeat_the_experiment(n_iterations, n_tosses)

x = mean(abs(n_heads_each_experiment - 125*2) >= observed_deviation)
paste0('With a fair coin we would expect a deviation as large or larger than what we observed ', x*100, '% of times')

[1] "With a fair coin we would expect a deviation as large or larger than what we observed 1.45% of times"

<a id="extrap2"></a>

In [26]:
# 1000
# Let's do 10,000 experiments of 1000 tosses and see how often our outcome deviates from what was actually observed (556 heads)

n_iterations = 10000
n_tosses = 250*4
observed_deviation = 139*4 - 125*4 # 556 heads observed, expected value for a fair coin of 500

n_heads_each_experiment = repeat_the_experiment(n_iterations, n_tosses)

x = mean(abs(n_heads_each_experiment - 125*4) >= observed_deviation)
paste0('With a fair coin we would expect a deviation as large or larger than what we observed ', x*100, '% of times')

[1] "With a fair coin we would expect a deviation as large or larger than what we observed 0.04% of times"

<a id="ans"></a>

**Do you think the coin was fair?** (Answer below in a markdown cell.)

***
While within the realm of possibility, the likely hood that a fair coin would deviate from the 50/50 is lower than 10%, and that is with repeating the experiment 10,000 times. The in the article the statiscs professor said less than 7%, which is a bit on the hyperbole side of reporting as running the code multiple times I never got a number lower than 8% (it seemed to hover within the range of 8.1%-9.6%).

<br>

If this was happening regularly enough to individuals to take notice and run an experiment themselves, I would say that <span style="color:#ff5e54">**it is unfair**</span>. 

<br>

Just to drive home this point, I wanted to see what would occur if the data sample was extrapolated (linear) to allow for more coin tosses than the 250 performed by the article. 

<br>

There is certainly a trend that shows a decrease in likelyhood we would see this deviation occur 20.73% -> <span style="color:#ff5e54">**8.28%**</span> -> 1.45% -> 0.04% for [125](#interp1), [<span style="color:#ff5e54">**250**](#org_exp)</span>, [500](#extrap1), and [1000](#extrap2) tosses respectively. However, 20% doesn't seem *that* unreasonable and this was for only 125 tosses, which is likely far more than the number of coin tosses preformed that meet the conditions below.

<br>

As you would need a large sample size to begin to show the likelyhood of this coin being unfair, it makes sense for how long this went unnoticed. You would need to:
1. Have at least if not more than 125 games come up to a coin toss 
2. Have those games be against (or in) Belgium 
3. Be using a coin minted in the set from Brussel (stated by the article)
